<a href="https://colab.research.google.com/github/ArindamBanerji/wip-experiments/blob/master/nlp/ner_NER_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
tf.__version__

'2.9.2'

In [2]:
tfds.__version__

'4.6.0'

In [3]:
# for the time  being comment out this piece 

# import wget
url_download_from = "https://gmb.let.rug.nl/releases/gmb-2.2.0.zip"

# !wget https://gmb.let.rug.nl/releases/gmb-2.2.0.zip
# wget.download('Url', 'C:\\PathToMyDownloadFolder\\NewFileName.extension')
# wget.download(url_download_from, ".")

In [4]:
import os
import sys
import shutil
import pickle

In [5]:
IN_COLAB = 'google.colab' in sys.modules
FNAMES_SAVED = False

print ("Runnig inn Google Colab :", IN_COLAB)

Runnig inn Google Colab : True


In [6]:
if (IN_COLAB) :
    ######## GPU CONFIGS FOR RTX 2070 ###############
    ## Please ignore if not training on GPU       ##
    ## this is important for running CuDNN on GPU ##

    tf.keras.backend.clear_session() #- for easy reset of notebook state

    # chck if GPU can be seen by TF
    tf.config.list_physical_devices('GPU')
    # only if you want to see how commands are executed, uncomment below
    # tf.debugging.set_log_device_placement(True)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
      # Restrict TensorFlow to only use the first GPU
      try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
      except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
    ###############################################


1 Physical GPUs, 1 Logical GPU


In [7]:
if (IN_COLAB) :
    from google.colab import drive

In [8]:
if (IN_COLAB) :
    mount_base = "/content/drive/My Drive"
    # Load the Drive helper and mount
  # This will prompt for authorization.
    drive.mount('/content/drive')
    mount_path = "/content/drive/My Drive/NLP/NER_EXPTS"
    ner_dirnm = mount_path + "/"
else :
    mount_base = "C:\\Users\\Arindam Banerji\\CopyFolder\\IOT_thoughts\\python-projects"
    mount_path = mount_base + "\\NLP\\NER_EXPTS"
    ner_dirnm = "G:\\My Drive\\NLP\\NER_EXPTS\\"

print (mount_path)
print (ner_dirnm)

Mounted at /content/drive
/content/drive/My Drive/NLP/NER_EXPTS
/content/drive/My Drive/NLP/NER_EXPTS/


In [9]:
path_exists = os.path.exists(mount_path)
ner_dir_exists = os.path.exists(ner_dirnm)

if path_exists : 
  print (mount_path, " : " , path_exists )
else:
  print ( "Load correct dir ", mount_path )

/content/drive/My Drive/NLP/NER_EXPTS  :  True


In [10]:
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

# Change the current working directory
os.chdir(mount_path)

# Print the current working directory
print("Changed working directory: {0}".format(os.getcwd()))

Current working directory: /content
Changed working directory: /content/drive/My Drive/NLP/NER_EXPTS


In [11]:
data_root = './gmb-2.2.0/data/' if IN_COLAB == True else '.\\gmb-2.2.0\\data\\' 

print ( data_root) 

./gmb-2.2.0/data/


In [12]:
fnames_list_fnm = ner_dirnm + "fnames_list.data"

In [13]:
def save_list_to_file ( fnames_list_fnm, fnames_list) :
        with open(fnames_list_fnm, 'wb') as filehandle:
            # Store the data as a binary data stream
            pickle.dump(fnames_list, filehandle)

In [14]:
def load_list_from_file ( fnames_list_fnm) :
    fnames_loaded = []
    if os.path.isfile(fnames_list_fnm) and os.access(fnames_list_fnm, os.R_OK) :
        with open(fnames_list_fnm, 'rb') as filehandle:
            # Read the data as a binary data stream
            fnames_loaded = pickle.load(filehandle)
    return fnames_loaded

In [15]:
# data_root = './gmb-2.2.0/data/'

fnames = []

if os.path.isfile(fnames_list_fnm) and os.access(fnames_list_fnm, os.R_OK) :
    fnames = load_list_from_file (fnames_list_fnm)
else :
        for root, dirs, files in os.walk(data_root):
            for filename in files:
                if filename.endswith(".tags"):
                    fnames.append(os.path.join(root, filename))
        
        save_list_to_file (fnames_list_fnm, fnames)
        FNAMES_SAVED = True

In [16]:
if FNAMES_SAVED :
    fnames_reloaded = load_list_from_file (fnames_list_fnm)
    print ("Are fnames && fnames_reloaded equal : ", (fnames == fnames_reloaded) )
    print ("FNAMES were saved to  ", fnames_list_fnm)
else :
    print ("FNAMES were loaded from ", fnames_list_fnm)

FNAMES were loaded from  /content/drive/My Drive/NLP/NER_EXPTS/fnames_list.data


In [17]:
print (" num of files ", len(fnames) )
fnames[:2]

 num of files  10000


['.\\gmb-2.2.0\\data\\p00\\d0018\\en.tags',
 '.\\gmb-2.2.0\\data\\p00\\d0025\\en.tags']

In [18]:
# save_list_to_file (fnames_list_fnm, fnames)
# fnames_loaded = load_list_from_file (fnames_list_fnm)
# print ("fnames is equal to fnames_loaded ", (fnames == fnames_loaded))

In [19]:
ner_path = "./ner"

ner_exists = os.path.exists(ner_path)
num_files_in_ner = 0

if ner_exists :
    onlyfiles = next(os.walk(ner_path))[2]
    num_files_in_ner = len(onlyfiles) 
    print ( len(onlyfiles) ) 
    shutil.rmtree(ner_path)
               
os.mkdir(ner_path)

print ("Directory status ", ner_path, " : ", os.path.exists(ner_path) )

Directory status  ./ner  :  True


In [20]:
import csv
import collections
 
ner_tags = collections.Counter()
iob_tags = collections.Counter()

def strip_ner_subcat(tag):
    # NER tags are of form {cat}-{subcat}
    # eg tim-dow. We only want first part
    return tag.split("-")[0]


def iob_format(ners):
    # converts IO tags into BIO format
    # input is a sequence of IO NER tokens
    # convert this: O, PERSON, PERSON, O, O, LOCATION, O
    # into: O, B-PERSON, I-PERSON, O, O, B-LOCATION, O
    iob_tokens = []
    for idx, token in enumerate(ners):
        if token != 'O':  # !other
            if idx == 0:
                token = "B-" + token #start of sentence
            elif ners[idx-1] == token:
                token = "I-" + token  # continues
            else:
                token = "B-" + token
        iob_tokens.append(token)
        iob_tags[token] += 1
    return iob_tokens  


In [21]:
total_sentences = 0
outfiles = []
for idx, file in enumerate(fnames):
    with open(file, 'rb') as content:
        data = content.read().decode('utf-8').strip()
        sentences = data.split("\n\n")
        print(idx, file, len(sentences))
        total_sentences += len(sentences)
        
        with open("./ner/"+str(idx)+"-"+os.path.basename(file), 
                  'w', encoding='utf-8') as outfile:
            outfiles.append("./ner/"+str(idx)+"-"+os.path.basename(file))
            writer = csv.writer(outfile)
            
            for sentence in sentences: 
                toks = sentence.split('\n')
                words, pos, ner = [], [], []
                
                for tok in toks:
                    t = tok.split("\t")
                    words.append(t[0])
                    pos.append(t[1])
                    ner_tags[t[3]] += 1
                    ner.append(strip_ner_subcat(t[3]))
                writer.writerow([" ".join(words), 
                                 " ".join(iob_format(ner)), 
                                 " ".join(pos)])

FileNotFoundError: ignored

In [ ]:
print("total number of sentences: ", total_sentences)

In [ ]:
print(" nertags : ", ner_tags, "\n")
print("iob tags ", iob_tags, "\n")

In [ ]:
import matplotlib.pyplot as plt

labels, values = zip(*iob_tags.items())

In [ ]:
indexes = np.arange(len(labels))


plt.bar(indexes, values)
plt.xticks(indexes, labels, rotation='vertical')
plt.margins(0.01)
plt.subplots_adjust(bottom=0.15)
plt.show()

In [ ]:
import glob
import pandas as pd

# could use `outfiles` param as well
files = glob.glob("./ner/*.tags")

data_pd = pd.concat([pd.read_csv(f, header=None, 
                                 names=["text", "label", "pos"]) 
                for f in files], ignore_index = True)

In [ ]:
data_pd.info()

In [ ]:
### Keras tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
text_tok = Tokenizer(filters='[\\]^\t\n', lower=False,
                     split=' ', oov_token='<OOV>')

pos_tok = Tokenizer(filters='\t\n', lower=False,
                    split=' ', oov_token='<OOV>')

ner_tok = Tokenizer(filters='\t\n', lower=False,
                    split=' ', oov_token='<OOV>')

In [ ]:
text_tok.fit_on_texts(data_pd['text'])
pos_tok.fit_on_texts(data_pd['pos'])
ner_tok.fit_on_texts(data_pd['label'])

In [ ]:
ner_config = ner_tok.get_config()
text_config = text_tok.get_config()

In [ ]:
print(ner_config)

In [ ]:
text_vocab = eval(text_config['index_word'])
ner_vocab = eval(ner_config['index_word'])

print("Unique words in vocab:", len(text_vocab))
print("Unique NER tags in vocab:", len(ner_vocab))

In [ ]:
x_tok = text_tok.texts_to_sequences(data_pd['text'])
y_tok = ner_tok.texts_to_sequences(data_pd['label'])

In [ ]:
print(text_tok.sequences_to_texts([x_tok[1]]), data_pd['text'][1])
print(ner_tok.sequences_to_texts([y_tok[1]]), data_pd['label'][1])

In [ ]:
print(text_tok.sequences_to_texts([x_tok[2]]), data_pd['text'][1])
print(ner_tok.sequences_to_texts([y_tok[2]]), data_pd['label'][1])

In [ ]:
# now, pad seqences to a maximum length
from tensorflow.keras.preprocessing import sequence

max_len = 50

x_pad = sequence.pad_sequences(x_tok, padding='post',
                              maxlen=max_len)
y_pad = sequence.pad_sequences(y_tok, padding='post',
                              maxlen=max_len)

In [ ]:
print(x_pad.shape, y_pad.shape)

In [ ]:
text_tok.sequences_to_texts([x_pad[4]])

In [ ]:
ner_tok.sequences_to_texts([y_pad[4]])

In [ ]:
num_classes = len(ner_vocab)+1

Y = tf.keras.utils.to_categorical(y_pad, num_classes=num_classes)
Y.shape

In [ ]:
# Length of the vocabulary 
vocab_size = len(text_vocab) + 1 

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 100

#batch size
BATCH_SIZE=90

# num of NER classes
num_classes = len(ner_vocab)+1

from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense

dropout=0.2
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size, classes):
  model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    Bidirectional(LSTM(units=rnn_units,
                           return_sequences=True,
                           dropout=dropout,  
                           kernel_initializer=tf.keras.initializers.he_normal())),
    TimeDistributed(Dense(rnn_units, activation='relu')),
    Dense(num_classes, activation="softmax")
  ])

  
  return model

In [ ]:
model = build_model_bilstm(
                        vocab_size = vocab_size,
                        embedding_dim=embedding_dim,
                        rnn_units=rnn_units,
                        batch_size=BATCH_SIZE,
                        classes=num_classes)
model.summary()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
X = x_pad 

In [ ]:
# create training and testing splits
total_sentences = 62010
test_size = round(total_sentences / BATCH_SIZE * 0.2)
X_train = X[BATCH_SIZE*test_size:]
Y_train = Y[BATCH_SIZE*test_size:]

X_test = X[0:BATCH_SIZE*test_size]
Y_test = Y[0:BATCH_SIZE*test_size]

In [ ]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=5)

In [ ]:
# batch size in eval
model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE)

In [ ]:
y_pred = model.predict(X_test, batch_size=BATCH_SIZE)

In [ ]:
text_tok.sequences_to_texts([X_test[4]])

In [ ]:
ner_tok.sequences_to_texts([y_pad[4]])

In [ ]:
y_pred = tf.argmax(y_pred, -1)
y_pred.shape

In [ ]:
y_pnp = y_pred.numpy()

In [ ]:
ner_tok.sequences_to_texts([y_pnp[4]])